<a href="https://colab.research.google.com/github/acesur/Machine-Learning-/blob/main/app_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# app.py
from flask import Flask, render_template, request, redirect, url_for
import os
from werkzeug.utils import secure_filename
import cv2
import numpy as np
import joblib
import pandas as pd
from utils import extract_abcd_features_robust

UPLOAD_FOLDER = 'static/uploads'
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

# Load trained model
model = joblib.load('model/classifier.pkl')
scaler = joblib.load('model/scaler.pkl')

# Image validation
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/predict', methods=['POST'])
def predict():
    if 'image' not in request.files:
        return redirect(request.url)

    file = request.files['image']
    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)

        features = extract_abcd_features_robust(filepath)
        if features is None:
            return render_template('result.html', error="Feature extraction failed.")

        input_df = pd.DataFrame([features])
        input_scaled = scaler.transform(input_df)
        pred_class = model.predict(input_scaled)[0]
        pred_proba = model.predict_proba(input_scaled).max()

        return render_template('result.html',
                               prediction=pred_class,
                               probability=round(pred_proba * 100, 2),
                               image_path=filepath)
    return redirect(url_for('index'))

if __name__ == '__main__':
    app.run(debug=True)


ModuleNotFoundError: No module named 'utils'